In [1]:
#Designate your file locations:

attributes_data_location=r'C:/Users/Jaketosis/OneDrive/TAMU/MS Stats/STAT626/Project/Data/Inputs/'
daily_data_location=r'C:/Users/Jaketosis/OneDrive/TAMU/MS Stats/STAT626/Project/Data/Inputs/'
intermediate_output_location=r'C:/Users/Jaketosis/OneDrive/TAMU/MS Stats/STAT626/Project/Data/Intermediate/'
aggregated_output_location=r'C:/Users/Jaketosis/OneDrive/TAMU/MS Stats/STAT626/Project/Data/Intermediate/'

#Designate your file names: 

attributes_name='Attributes_DataFrame.csv'
daily_name='Daily_DataFrame.csv'
intermediate_output_name='transformed_data.csv'
total_aggregated_output_name='aggregated_daily.csv'

In [2]:
import pandas as pd
import numpy as np
import cpi

In [ ]:
#Only run this if import cpi gives you a warning
#cpi.update()

In [ ]:
attributes_data=pd.read_csv(attributes_data_location+attributes_name)
daily_data=pd.read_csv(daily_data_location+daily_name)

In [ ]:
daily_data['year']=daily_data['Date'].str[:4].astype(int)
daily_data['month']=daily_data['Date'].str[5:7].astype(int)
daily_data

,Movie_Title,Date,Daily,Theaters,Rank,year,month
0,Jurassic World (2015),2015-06-12,81953950,4274.0,1,2015,6
1,Jurassic World (2015),2015-06-13,69644830,4274.0,1,2015,6
2,Jurassic World (2015),2015-06-14,57207490,4274.0,1,2015,6
3,Jurassic World (2015),2015-06-15,25344820,4274.0,1,2015,6
4,Jurassic World (2015),2015-06-16,24342515,4274.0,1,2015,6
...,...,...,...,...,...,...,...
254057,A Call to Spy (2020),2020-11-17,623,8.0,17,2020,11
254058,A Call to Spy (2020),2020-11-18,568,8.0,17,2020,11
254059,A Call to Spy (2020),2020-11-19,597,8.0,17,2020,11
254060,Love Story (2020),2020-02-09,87198,650.0,20,2020,2


In [ ]:
split=attributes_data.Genres.str.split(';',expand=True)
split.columns=["Genre "+str(colname) for colname in list(split.columns)]
list(split.columns)
attributes_data=attributes_data.join(split)

In [ ]:
merged_data=daily_data.merge(attributes_data,how='left',left_on='Movie_Title',right_on='Title')
merged_data['daily_pre_adjust']=merged_data['Daily']

In [ ]:
from datetime import date

to_transform_years=merged_data['year'].to_list()
to_transform_months=merged_data['month'].to_list()
to_tranform_vals=merged_data['Daily'].to_list()

new_list=[]
loop_list=[(to_transform_years[j], to_transform_months[j],to_tranform_vals[j]) for j in range(0,len(to_tranform_vals))]
for i_year,i_month,i in loop_list:
    append=cpi.inflate(i,date(i_year,i_month,1),to=date(2022,12,31))
    new_list=new_list+[append]

pd.Series(new_list)

0         1.019271e+08
1         8.661813e+07
2         7.114966e+07
3         3.152166e+07
4         3.027508e+07
              ...     
254057    7.105454e+02
254058    6.478167e+02
254059    6.808919e+02
254060    1.000476e+05
254061    7.607691e+04
Length: 254062, dtype: float64

In [ ]:
merged_data['Daily']=pd.Series(new_list)

In [ ]:
#We want an intermediate output of our data before aggregating for use in R or a visualization software
merged_data.to_csv(intermediate_output_location+intermediate_output_name)

In [ ]:
#merged_data.head()
merged_data.tail()

,Movie_Title,Date,Daily,Theaters,Rank,year,month,Title,Domestic,International,...,Genre 0,Genre 1,Genre 2,Genre 3,Genre 4,Genre 5,Genre 6,Genre 7,Genre 8,daily_pre_adjust
254057,A Call to Spy (2020),2020-11-17,710.545447,8.0,17,2020,11,A Call to Spy (2020),159014,681026,...,Biography,Crime,Drama,Thriller,War,None,None,None,None,623
254058,A Call to Spy (2020),2020-11-18,647.816715,8.0,17,2020,11,A Call to Spy (2020),159014,681026,...,Biography,Crime,Drama,Thriller,War,None,None,None,None,568
254059,A Call to Spy (2020),2020-11-19,680.891864,8.0,17,2020,11,A Call to Spy (2020),159014,681026,...,Biography,Crime,Drama,Thriller,War,None,None,None,None,597
254060,Love Story (2020),2020-02-09,100047.568042,650.0,20,2020,2,Love Story (2020),153504,153504,...,Drama,Romance,None,None,None,None,None,None,None,87198
254061,Love Story (2020),2020-02-12,76076.905968,650.0,18,2020,2,Love Story (2020),153504,153504,...,Drama,Romance,None,None,None,None,None,None,None,66306


In [ ]:
#Check the range of the dates in the dataset. Notice that June 2021 was way too early for Movie theaters to be recovering during COVID. 
print(min(pd.to_datetime(merged_data['Date'])),max(pd.to_datetime(merged_data['Date'])))

1999-08-06 00:00:00 2021-06-24 00:00:00


In [ ]:
aggregated_data=merged_data.groupby(['Date']).agg({'Daily':'max'})
aggregated_data['min']=merged_data.groupby(['Date']).agg({'Daily':'min'})
aggregated_data['mean']=merged_data.groupby(['Date']).agg({'Daily':'mean'})
aggregated_data['sum']=merged_data.groupby(['Date']).agg({'Daily':'sum'})
aggregated_data.rename(columns={'Daily':'max'},inplace=True)
aggregated_data.reset_index(inplace=True)
aggregated_data['Date']=pd.to_datetime(aggregated_data['Date'])

#Write out your code to be consumed in R or a visualization software
aggregated_data.loc[aggregated_data['Date'].dt.year>2000].to_csv(aggregated_output_location+total_aggregated_output_name)

In [ ]:
aggregated_data

,Date,max,min,mean,sum
0,1999-08-06,1.421819e+07,1.421819e+07,1.421819e+07,1.421819e+07
1,1999-08-07,1.713998e+07,1.713998e+07,1.713998e+07,1.713998e+07
2,1999-08-08,1.545085e+07,1.545085e+07,1.545085e+07,1.545085e+07
3,1999-08-09,7.726867e+06,7.726867e+06,7.726867e+06,7.726867e+06
4,1999-08-10,7.852726e+06,7.852726e+06,7.852726e+06,7.852726e+06
...,...,...,...,...,...
7954,2021-06-20,1.361660e+04,1.361660e+04,1.361660e+04,1.361660e+04
7955,2021-06-21,6.707252e+03,6.707252e+03,6.707252e+03,6.707252e+03
7956,2021-06-22,6.090054e+03,6.090054e+03,6.090054e+03,6.090054e+03
7957,2021-06-23,6.412308e+03,6.412308e+03,6.412308e+03,6.412308e+03
